# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [86]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

## Split the Data into Training and Testing

In [70]:
# Create our features
X = df.copy()
X.drop(columns=['loan_status','issue_d','next_pymnt_d'], axis=1, inplace=True)

# Create our target
y = df['loan_status']

In [71]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [33]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [82]:
# Find columns that have string values
for name, values in df.iteritems():    
    print(name)
    print(type(values[0]))

loan_amnt
<class 'numpy.float64'>
int_rate
<class 'numpy.float64'>
installment
<class 'numpy.float64'>
home_ownership
<class 'str'>
annual_inc
<class 'numpy.float64'>
verification_status
<class 'str'>
issue_d
<class 'str'>
loan_status
<class 'str'>
pymnt_plan
<class 'str'>
dti
<class 'numpy.float64'>
delinq_2yrs
<class 'numpy.float64'>
inq_last_6mths
<class 'numpy.float64'>
open_acc
<class 'numpy.float64'>
pub_rec
<class 'numpy.float64'>
revol_bal
<class 'numpy.float64'>
total_acc
<class 'numpy.float64'>
initial_list_status
<class 'str'>
out_prncp
<class 'numpy.float64'>
out_prncp_inv
<class 'numpy.float64'>
total_pymnt
<class 'numpy.float64'>
total_pymnt_inv
<class 'numpy.float64'>
total_rec_prncp
<class 'numpy.float64'>
total_rec_int
<class 'numpy.float64'>
total_rec_late_fee
<class 'numpy.float64'>
recoveries
<class 'numpy.float64'>
collection_recovery_fee
<class 'numpy.float64'>
last_pymnt_amnt
<class 'numpy.float64'>
next_pymnt_d
<class 'str'>
collections_12_mths_ex_med
<class 'nu

In [72]:
X_encoded = pd.get_dummies(X, columns=["home_ownership", "verification_status","initial_list_status","application_type"])

In [73]:
X_encoded['pymnt_plan'] = X_encoded['pymnt_plan'].map({'y': 1, 'n': 0})

In [74]:
X_encoded['hardship_flag'] = X_encoded['hardship_flag'].map({'Y': 1, 'N': 0})

In [75]:
X_encoded['debt_settlement_flag'] = X_encoded['debt_settlement_flag'].map({'Y': 1, 'N': 0})

In [76]:
X_encoded

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,0,27.24,0.0,0.0,8.0,0.0,...,0,0,1,0,1,0,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,0,20.23,0.0,0.0,17.0,1.0,...,1,0,0,0,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,0,24.26,0.0,0.0,8.0,0.0,...,1,0,0,0,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,0,31.44,0.0,1.0,10.0,1.0,...,0,0,1,0,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,0,18.76,0.0,1.0,14.0,0.0,...,1,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,0,9.60,0.0,0.0,9.0,0.0,...,0,0,1,0,1,0,0,1,1,0
68813,12000.0,0.2727,368.37,63000.0,0,29.07,0.0,0.0,8.0,0.0,...,0,0,1,1,0,0,0,1,1,0
68814,5000.0,0.1992,185.62,52000.0,0,14.86,0.0,0.0,5.0,1.0,...,1,0,0,0,1,0,0,1,1,0
68815,40000.0,0.0646,1225.24,520000.0,0,9.96,0.0,1.0,21.0,0.0,...,1,0,0,0,0,1,1,0,1,0


In [77]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [78]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [79]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [80]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [93]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

y_pred_rf = brf.predict(X_test_scaled)

In [99]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, y_pred_rf)
acc_score

0.8220284800929962

In [95]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_rf)

array([[   62,    39],
       [ 3023, 14081]], dtype=int64)

In [96]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.61      0.82      0.04      0.71      0.49       101
   low_risk       1.00      0.82      0.61      0.90      0.71      0.52     17104

avg / total       0.99      0.82      0.62      0.90      0.71      0.52     17205



In [98]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_

sorted(zip(brf.feature_importances_, X_encoded.columns), reverse=True)

[(0.07494048315251686, 'total_rec_prncp'),
 (0.07260965739109633, 'total_pymnt_inv'),
 (0.05612247909175081, 'total_rec_int'),
 (0.049427435108159105, 'last_pymnt_amnt'),
 (0.04433909177189386, 'total_pymnt'),
 (0.02839390736477029, 'int_rate'),
 (0.01982973769574149, 'dti'),
 (0.019470410962588624, 'installment'),
 (0.01787594776837803, 'out_prncp_inv'),
 (0.01787119029813518, 'mths_since_rcnt_il'),
 (0.017617556025036715, 'revol_bal'),
 (0.017575972819394155, 'max_bal_bc'),
 (0.017125720519389303, 'bc_open_to_buy'),
 (0.017049157180886013, 'total_rev_hi_lim'),
 (0.017045994552396696, 'total_bal_ex_mort'),
 (0.016361338439176813, 'annual_inc'),
 (0.016114231211331376, 'out_prncp'),
 (0.015749565892644004, 'mo_sin_old_il_acct'),
 (0.015658449415388614, 'avg_cur_bal'),
 (0.015412543367225051, 'il_util'),
 (0.01537621031006064, 'total_il_high_credit_limit'),
 (0.014278033234559638, 'tot_hi_cred_lim'),
 (0.013962527087031513, 'total_bal_il'),
 (0.013896975320537552, 'total_bc_limit'),
 (0

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
# YOUR CODE HERE

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.